In [4]:

import pandas as pd 
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from nba_api.stats.endpoints import commonallplayers

In [7]:
all_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]

In [8]:
# change data type
all_players['TO_YEAR'] = all_players['TO_YEAR'].astype('int64')

In [9]:
# filter players that have been played during season 2000 to 2020
all_players = all_players[all_players['TO_YEAR']>=2000]

In [10]:
from nba_api.stats.endpoints import playercareerstats
import time

In [11]:
# extract all IDs
players_ID = all_players['PERSON_ID']

In [10]:
# initial an empty dataframe
players_stats = pd.DataFrame()
# save fail queries
error_log = []

In [27]:
# extracting stats
for ID in tqdm(players_ID):
    try:
        time.sleep(1) # avoid too many queries submitted at the same time
        career = playercareerstats.PlayerCareerStats(player_id=ID)
        player_career = career.get_data_frames()[0]
        players_stats = pd.concat([players_stats,player_career],axis=0,ignore_index=True)
    except:
        error_log.append(ID)

  0%|          | 0/2217 [00:03<?, ?it/s]


NameError: name 'error_log' is not defined

In [14]:
# re-query missing IDs 
for ID in tqdm(error_log):
    try:
        time.sleep(1)
        career = playercareerstats.PlayerCareerStats(player_id=ID)
        player_career = career.get_data_frames()[0]
        players_stats = pd.concat([players_stats,player_career],axis=0,ignore_index=True)
    except:
        error_log.append(ID)

0it [00:00, ?it/s]


In [26]:
with open("player_stats.pkl",'wb') as f:
    pickle.dump(players_stats,f)




NameError: name 'players_stats' is not defined

In [1]:
from nba_api.stats.endpoints import teamyearbyyearstats
from nba_api.stats.static import teams

In [2]:
nba_teams = teams.get_teams()

In [5]:
teams_stats = pd.DataFrame()
error_teams =[]
for i in tqdm(nba_teams):
    try:
        time.sleep(1)
        team = teamyearbyyearstats.TeamYearByYearStats(team_id=i['id'])
        team_data = team.get_data_frames()[0]
        teams_stats = pd.concat([teams_stats,team_data],axis=0,ignore_index=True)
    except:
        error_teams.append(i)

100%|██████████| 30/30 [00:00<?, ?it/s]


In [6]:
with open("teams_stats.pkl","wb") as f:
    pickle.dump(teams_stats,f)


In [9]:
teams_stats.to_csv("teams_stats.csv", sep='\t', encoding='utf-8')

In [28]:
from nba_api.stats.endpoints import playerawards

In [29]:
players_awards = pd.DataFrame()
error_awards = []
for ID in tqdm(players_ID):
    try:
        time.sleep(1)
        award = playerawards.PlayerAwards(player_id=ID)
        award.get_data_frames()[0]
        players_awards = pd.concat([players_awards,award.get_data_frames()[0]],axis=0,ignore_index=True)
    except:
        error_awards.append(ID)

  1%|          | 16/2217 [00:28<1:02:13,  1.70s/it]

In [15]:
for ID in tqdm(error_awards):
    time.sleep(1)
    award = playerawards.PlayerAwards(player_id=ID)
    award.get_data_frames()[0]
    players_awards = pd.concat([players_awards,award.get_data_frames()[0]],axis=0,ignore_index=True)

0it [00:00, ?it/s]


In [16]:
with open("players_awards.pkl","wb") as f:
    pickle.dump(players_awards,f)